## 僅用於統一兩邊的title, 然後改變各自原本title_id成為新的號碼 回傳新的df

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/MyDrive/Colab/資料分析期末') #切換該目錄
os.listdir() #確認目錄內容

Mounted at /content/drive


['data',
 'models',
 'preprocessed_output',
 '過往東東',
 'share_data',
 'model選特徵0601.ipynb',
 'submit.csv',
 'LSTM_torch_duration.ipynb',
 'train的其餘特徵.ipynb',
 'test其餘特徵處理.ipynb',
 'unify_title.ipynb']

In [2]:
# always needed
import math, os, random, csv
import pandas as pd
import numpy as np

# log and save
import json, logging, pickle, sys, shutil, copy
# torch
import torch
import torch.nn
from torch.nn import Conv2d, MaxPool2d, Flatten, Linear, ReLU
import torchvision
from tqdm.auto import tqdm
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader,ConcatDataset
from torchvision import datasets, models, transforms
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter
%matplotlib inline
import seaborn as sns

# others
import matplotlib.pyplot as plt
from PIL import Image

# sklearn
from sklearn import preprocessing

# statistics
from scipy import stats
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statistics

# seeds
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


In [3]:
df_test = pd.read_csv("./data/test_source_events.csv")
df_train = pd.read_csv("./data/train_source_events.csv")
df = pd.concat([df_test, df_train])

In [4]:
title_duration_sum = df.groupby('title_id')['played_duration'].sum()
title_duration_list = list(title_duration_sum.items())
sorted_title_duration = sorted(title_duration_list, key=lambda x: x[1], reverse=True)
sorted_title_ids = [title for title, _ in sorted_title_duration] #拿到是哪些title被觀看最久
print(sorted_title_ids[:10])

[47, 9, 96, 17, 68, 4, 166, 37, 6, 11]


In [5]:
mapping = {}
now = len(df['title_id'].unique())
for i in sorted_title_ids:
  mapping[i] =now
  now -= 1

In [6]:
df_train['trans_title'] = df_train['title_id'].map(mapping)
df_train['trans_title'].value_counts()

6093    149946
6094    149189
6092    129222
6091    113745
6076    110886
         ...  
1225         1
58           1
922          1
160          1
339          1
Name: trans_title, Length: 6032, dtype: int64

In [15]:
df_train.to_csv('./data/new_train_source.csv', index=False)

In [9]:
df_test['title_id'].value_counts()

9       39093
96      31002
47      30209
17      27826
181     27688
        ...  
6126        1
5621        1
5199        1
6128        1
5211        1
Name: title_id, Length: 5336, dtype: int64

In [10]:
df_test['trans_title'] = df_test['title_id'].map(mapping)
df_test['trans_title'].value_counts()

6093    39093
6092    31002
6094    30209
6091    27826
6081    27688
        ...  
86          1
2800        1
681         1
142         1
894         1
Name: trans_title, Length: 5336, dtype: int64

In [12]:
df_test.to_csv('./data/new_test_source.csv', index=False)


In [13]:
loaded_df = pd.read_csv('./data/new_test_source.csv')

In [14]:
loaded_df

,user_id,device_id,session_id,title_id,event_time,played_duration,action_trigger,platform,title_in_simulcast,internet_connection_type,trans_title
0,30460,139780,4203642,134,1.644848e+09,919,0,0,1,1,6066
1,30460,139780,4203766,134,1.644854e+09,1,0,0,1,1,6066
2,30460,139780,4206734,331,1.645010e+09,22,0,0,1,0,5822
3,30460,139780,4217713,331,1.645705e+09,1220,1,0,1,0,5822
4,30460,139780,4193024,398,1.646645e+09,9,0,0,1,0,5995
...,...,...,...,...,...,...,...,...,...,...,...
2382746,38075,174948,5252063,625,1.662919e+09,4886,1,1,0,2,6017
2382747,38075,174948,5252063,625,1.662919e+09,5,0,1,0,2,6017
2382748,38075,174948,5252063,625,1.662919e+09,1,0,1,0,2,6017
2382749,38075,174948,5252064,625,1.663256e+09,4823,2,1,0,2,6017
